In [ ]:
import os
from matplotlib import pyplot as plt
import tensorflow as tf 
import tensorflow_io as tfio

In [ ]:
CAPUCHIN_FILE = os.path.join("data", "Parsed_Capuchinbird_Clips", "XC3776-3.wav")
NOT_CAPUCHIN_FILE = os.path.join("data", "Parsed_not_Capuchinbird_Clips", "afternoon-birds-song-in-forest-0.wav")

In [ ]:
def load_wav_16k_mono(filename):
    file_contents = tf.io.read_file(filename)
    wav, sr = tf.audio.decode_wav(file_contents, desired_channels=1)
    wav = tf.squeeze(wav, axis=-1)
    sr = tf.cast(sr, dtype=tf.int64)
    wav = tfio.audio.resample(wav, rate_in=sr, rate_out=16000)
    return wav

In [ ]:
wave = load_wav_16k_mono(CAPUCHIN_FILE)
nwave = load_wav_16k_mono(NOT_CAPUCHIN_FILE)

In [ ]:
plt.plot(wave)
plt.plot(nwave)

In [ ]:
POS = os.path.join("data", "Parsed_Capuchinbird_Clips")
NEG = os.path.join("data", "Parsed_not_Capuchinbird_Clips")

In [ ]:
pos = tf.data.Dataset.list_files(f"{POS}\*.wav")
neg = tf.data.Dataset.list_files(f"{NEG}\*.wav")

In [ ]:
pos.as_numpy_iterator().next()

In [ ]:
positives = tf.data.Dataset.zip((pos, tf.data.Dataset.from_tensor_slices(tf.ones(len(pos)))))
negatives = tf.data.Dataset.zip((neg, tf.data.Dataset.from_tensor_slices(tf.ones(len(neg)))))
data = positives.concatenate(negatives)

In [ ]:
# Find average capuchin call length
lengths = []
for file in os.listdir(POS):
    tensor_wave = load_wav_16k_mono(os.path.join(POS, file))
    lengths.append(len(tensor_wave))

In [ ]:
print(tf.math.reduce_mean(lengths))
print(tf.math.reduce_min(lengths))
print(tf.math.reduce_max(lengths))

In [ ]:
def preprocess(file_path, label):
    wav=load_wav_16k_mono(file_path)
    wav = wav[:48000]
    zero_padding = tf.zeros([48000] - tf.shape(wav), dtype=tf.float32)
    wav = tf.concat([zero_padding, wav], 0)
    spectrogram = tf.signal.stft(wav, frame_length=320, frame_step=32)
    spectrogram = tf.abs(spectrogram)
    spectrogram = tf.expand_dims(spectrogram, axis=2)
    return spectrogram, label

In [ ]:
filepath, label = positives.shuffle(10000).as_numpy_iterator().next()

In [ ]:
spectrogram, label = preprocess(filepath, label)

In [ ]:
plt.figure(figsize=(30,20))
plt.imshow(tf.transpose(spectrogram)[0],origin='lower')
plt.show()

CREATE TRAINING AND TESTING PARTITIONS

In [ ]:
data = data.map(preprocess)
data = data.cache()
data = data.shuffle(buffer_size=1000)
data = data.batch(16)
data = data.prefetch(8)

In [ ]:
train = data.take(36)
test = data.skip(36).take(15)

In [ ]:
samples, labels = train.as_numpy_iterator().next()
samples.shape

BUILD DEEP LEARNING MODEL

In [2]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, Dense, Flatten,MaxPooling2D,Dropout

In [3]:
model = Sequential([
    Conv2D(filters=64, kernel_size=(3,3), activation="relu" ,input_shape=(1491, 257,1)),
    MaxPooling2D(pool_size=2),
    Conv2D(filters=128, kernel_size=(3,3), activation="relu"),
    Conv2D(filters=128, kernel_size=(3,3), activation="relu"),
    MaxPooling2D(pool_size=2),
    Conv2D(filters=256, kernel_size=(3,3)),
    Conv2D(filters=256, kernel_size=(3,3)),
    MaxPooling2D(pool_size=2),
    Flatten(),
    Dense(units=128, activation='relu'),
    Dropout(0.5),
    Dense(units=64, activation='relu'),
    Dropout(0.5),
    Dense(units=1, activation='softmax'),
])

In [ ]:
model.compile("Adam", loss="BinaryCrossentropy", metrics=[tf.keras.metrics.Recall(), tf.keras.metrics.Precision(), tf.keras.metrics.Accuracy()])

In [4]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 1489, 255, 16)     160       
                                                                 
 conv2d_1 (Conv2D)           (None, 1487, 253, 16)     2320      
                                                                 
 flatten (Flatten)           (None, 6019376)           0         
                                                                 
 dense (Dense)               (None, 128)               770480256 
                                                                 
 dense_1 (Dense)             (None, 1)                 129       
                                                                 
Total params: 770,482,865
Trainable params: 770,482,865
Non-trainable params: 0
_________________________________________________________________


In [ ]:
hist = model.fit(train, epochs=4, validation_data=test)

In [1]:
def load_mp3_16k_mono(filename):
    """ Load a WAV file, convert it to a float tensor, resample to 16 kHz single-channel audio. """
    res = tfio.audio.AudioIOTensor(filename)
    # Convert to tensor and combine channels 
    tensor = res.to_tensor()
    tensor = tf.math.reduce_sum(tensor, axis=1) / 2 
    # Extract sample rate and cast
    sample_rate = res.rate
    sample_rate = tf.cast(sample_rate, dtype=tf.int64)
    # Resample to 16 kHz
    wav = tfio.audio.resample(tensor, rate_in=sample_rate, rate_out=16000)
    return wav


In [2]:
def preprocess_mp3(sample, index):
    sample = sample[0]
    zero_padding = tf.zeros([48000] - tf.shape(sample), dtype=tf.float32)
    wav = tf.concat([zero_padding, sample],0)
    spectrogram = tf.signal.stft(wav, frame_length=320, frame_step=32)
    spectrogram = tf.abs(spectrogram)
    spectrogram = tf.expand_dims(spectrogram, axis=2)
    return spectrogram

In [ ]:

X_test, y_test = test.as_numpy_iterator().next()
with tf.device("/gpu:0"):
    yhat = model.predict(X_test)

    yhat = [1 if prediction > 0.5 else 0 for prediction in yhat]

mp3 = os.path.join("data", "Forest Recordings", "recording_00.mp3")

wav = load_mp3_16k_mono(mp3)

audio_slices = tf.keras.utils.timeseries_dataset_from_array(wav, wav, sequence_length=48000, sequence_stride=48000, batch_size=1)

samples, index = audio_slices.as_numpy_iterator().next()

print(len(audio_slices))
print(samples.shape)

audio_slices = audio_slices.map(preprocess_mp3)
audio_slices = audio_slices.batch(8)

yhat = model.predict(audio_slices)
yhat = [1 if prediction > 0.5 else 0 for prediction in yhat]

